In [1]:
import gc
import os
import json
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    char_to_num = json.load(f)

df = pd.read_csv('/kaggle/input/asl-fingerspelling/train.csv')

LIP = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
NOSE = [1, 2, 98, 327]
LEYE = [466, 388, 387, 386, 385, 384, 398,
        263, 249, 390, 373, 374, 380, 381, 382, 362]
REYE = [246, 161, 160, 159, 158, 157, 173,
        33, 7, 163, 144, 145, 153, 154, 155, 133]
FACE = LIP + NOSE + LEYE + REYE

LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

X = [f'x_right_hand_{i}' for i in range(21)] + [f'x_left_hand_{i}' for i in range(21)] + [f'x_pose_{i}' for i in POSE] + [f'x_face_{i}' for i in FACE]
Y = [f'y_right_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'y_pose_{i}' for i in POSE] + [f'y_face_{i}' for i in FACE]
Z = [f'z_right_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)] + [f'z_pose_{i}' for i in POSE] + [f'z_face_{i}' for i in FACE]

SEL_COLS = X + Y + Z

LIP_IDX_X   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in LIP and "x" in col]
NOSE_IDX_X  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in NOSE and "x" in col]
LEYE_IDX_X  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in LEYE and "x" in col]
REYE_IDX_X  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in REYE and "x" in col]
RHAND_IDX_X = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "x" in col]
LHAND_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "x" in col]
RPOSE_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "x" in col]
LPOSE_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "x" in col]

LIP_IDX_Y   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in LIP and "y" in col]
NOSE_IDX_Y  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in NOSE and "y" in col]
LEYE_IDX_Y  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in LEYE and "y" in col]
REYE_IDX_Y  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in REYE and "y" in col]
RHAND_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "y" in col]
LHAND_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "y" in col]
RPOSE_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "y" in col]
LPOSE_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "y" in col]

LIP_IDX_Z   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in LIP and "z" in col]
NOSE_IDX_Z  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in NOSE and "z" in col]
LEYE_IDX_Z  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in LEYE and "z" in col]
REYE_IDX_Z  = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and int(col.split("_")[-1]) in REYE and "z" in col]
RHAND_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "z" in col]
LHAND_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "z" in col]
RPOSE_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "z" in col]
LPOSE_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "z" in col]

In [3]:
def load_relevant_data_subset(pq_path):
    return pd.read_parquet(pq_path, columns=SEL_COLS)

file_id = df.file_id.iloc[0]
inpdir = "/kaggle/input/asl-fingerspelling/train_landmarks"
pqfile = f"{inpdir}/{file_id}.parquet"
seq_refs = df.loc[df.file_id == file_id]
seqs = load_relevant_data_subset(pqfile)

seq_id = seq_refs.sequence_id.iloc[0]
frames = seqs.iloc[seqs.index == seq_id].to_numpy()
phrase = str(df.loc[df.sequence_id == seq_id].phrase.iloc[0])

In [4]:
def process(x):
    lip_x = x[:, LIP_IDX_X]
    lip_y = x[:, LIP_IDX_Y]
    lip_z = x[:, LIP_IDX_Z]

    nose_x = x[:, NOSE_IDX_X]
    nose_y = x[:, NOSE_IDX_Y]
    nose_z = x[:, NOSE_IDX_Z]
    
    leye_x = x[:, LEYE_IDX_X]
    leye_y = x[:, LEYE_IDX_Y]
    leye_z = x[:, LEYE_IDX_Z]
    
    reye_x = x[:, REYE_IDX_X]
    reye_y = x[:, REYE_IDX_Y]
    reye_z = x[:, REYE_IDX_Z]
    
    rhand_x = x[:, RHAND_IDX_X]
    rhand_y = x[:, RHAND_IDX_Y]
    rhand_z = x[:, RHAND_IDX_Z]
    
    lhand_x = x[:, LHAND_IDX_X]
    lhand_y = x[:, LHAND_IDX_Y]
    lhand_z = x[:, LHAND_IDX_Z]

    rpose_x = x[:, RPOSE_IDX_X]
    rpose_y = x[:, RPOSE_IDX_Y]
    rpose_z = x[:, RPOSE_IDX_Z]
    
    lpose_x = x[:, LPOSE_IDX_X]
    lpose_y = x[:, LPOSE_IDX_Y]
    lpose_z = x[:, LPOSE_IDX_Z]
    
    rhnonans = ~np.isnan(np.sum(rhand_x, axis=1))
    lhnonans = ~np.isnan(np.sum(lhand_x, axis=1))
    lpnonans = ~np.isnan(np.sum(lip_x, axis=1))
    nosenonans = ~np.isnan(np.sum(nose_x, axis=1))
    reyenonans = ~np.isnan(np.sum(reye_x, axis=1))
    leyenonans = ~np.isnan(np.sum(leye_x, axis=1))
    
    rhand = np.stack([rhand_x, rhand_y, rhand_z], axis=-1)[rhnonans]
    rpose = np.stack([rpose_x, rpose_y, rpose_z], axis=-1)[rhnonans]
    
    lhand = np.stack([lhand_x, lhand_y, lhand_z], axis=-1)[lhnonans]
    lpose = np.stack([lpose_x, lpose_y, lpose_z], axis=-1)[lhnonans]
    
    lip = np.stack([lip_x, lip_y, lip_z], axis=-1)[lpnonans]
    nose = np.stack([nose_x, nose_y, nose_z], axis=-1)[nosenonans]
    leye = np.stack([leye_x, leye_y, leye_z], axis=-1)[reyenonans]
    reye = np.stack([reye_x, reye_y, reye_z], axis=-1)[leyenonans]

    return rhand, lhand, rpose, lpose, lip, nose, leye, reye

rhand, lhand, rpose, lpose, lip, nose, leye, reye = process(frames)
print(rhand.shape, lhand.shape, rpose.shape, lpose.shape, lip.shape, nose.shape, leye.shape, reye.shape)

(58, 21, 3) (0, 21, 3) (58, 5, 3) (0, 5, 3) (123, 40, 3) (123, 4, 3) (123, 16, 3) (123, 16, 3)


In [5]:
def gen(df):
    for file_id in df.file_id.unique():
        pqfile = f"/kaggle/input/asl-fingerspelling/train_landmarks/{file_id}.parquet"
        seq_refs = df.loc[df.file_id == file_id]
        seqs = load_relevant_data_subset(pqfile)

        for seq_id in seq_refs.sequence_id:
            x = seqs.iloc[seqs.index == seq_id].to_numpy()
            y = df.loc[df.sequence_id == seq_id].phrase.iloc[0]
            rhand, lhand, rpose, lpose, lip, nose, leye, reye = process(x)
            
            if max(rhand.shape[0], lhand.shape[0]) > len(y):
                yield rhand, lhand, rpose, lpose, lip, nose, leye, reye

In [6]:
!mkdir mean_std
RHAND = []
LHAND = []
RPOSE = []
LPOSE = []
LIP = []
NOSE = []
LEYE = []
REYE = []

print("## processing rhand")
for rhand, lhand, rpose, lpose, lip, nose, leye, reye in tqdm(gen(df)):
    RHAND.extend(rhand)
RHAND = np.array(RHAND)
rh_mean = np.mean(RHAND, axis=0)
rh_std = np.std(RHAND, axis=0)
np.save("mean_std/rh_mean.npy", rh_mean)
np.save("mean_std/rh_std.npy", rh_std)
del RHAND, rh_mean, rh_std
_ = gc.collect()

print("## processing lhand")
for rhand, lhand, rpose, lpose, lip, nose, leye, reye in tqdm(gen(df)):
    LHAND.extend(lhand)
LHAND = np.array(LHAND)
lh_mean = np.mean(LHAND, axis=0)
lh_std = np.std(LHAND, axis=0)
np.save("mean_std/lh_mean.npy", lh_mean)
np.save("mean_std/lh_std.npy", lh_std)
del LHAND, lh_mean, lh_std
_ = gc.collect()

print("## processing rpose")
for rhand, lhand, rpose, lpose, lip, nose, leye, reye in tqdm(gen(df)):
    RPOSE.extend(rpose)
RPOSE = np.array(RPOSE)
rp_mean = np.mean(RPOSE, axis=0)
rp_std = np.std(RPOSE, axis=0)
np.save("mean_std/rp_mean.npy", rp_mean)
np.save("mean_std/rp_std.npy", rp_std)
del RPOSE, rp_mean, rp_std
_ = gc.collect()

print("## processing lpose")
for rhand, lhand, rpose, lpose, lip, nose, leye, reye in tqdm(gen(df)):
    LPOSE.extend(lpose)
LPOSE = np.array(LPOSE)
lp_mean = np.mean(LPOSE, axis=0)
lp_std = np.std(LPOSE, axis=0)
np.save("mean_std/lp_mean.npy", lp_mean)
np.save("mean_std/lp_std.npy", lp_std)
del LPOSE, lp_mean, lp_std
_ = gc.collect()

print("## processing lip")
for rhand, lhand, rpose, lpose, lip, nose, leye, reye in tqdm(gen(df)):
    LIP.extend(lip)
LIP = np.array(LIP)
lip_mean = np.mean(LIP, axis=0)
lip_std = np.std(LIP, axis=0)
np.save("mean_std/lip_mean.npy", lip_mean)
np.save("mean_std/lip_std.npy", lip_std)
del LIP, lip_mean, lip_std
_ = gc.collect()

print("## processing nose")
for rhand, lhand, rpose, lpose, lip, nose, leye, reye in tqdm(gen(df)):
    NOSE.extend(nose)
NOSE = np.array(NOSE)
nose_mean = np.mean(NOSE, axis=0)
nose_std = np.std(NOSE, axis=0)
np.save("mean_std/nose_mean.npy", nose_mean)
np.save("mean_std/nose_std.npy", nose_std)
del NOSE, nose_mean, nose_std
_ = gc.collect()

print("## processing leye")
for rhand, lhand, rpose, lpose, lip, nose, leye, reye in tqdm(gen(df)):
    LEYE.extend(leye)
LEYE = np.array(LEYE)
leye_mean = np.mean(LEYE, axis=0)
leye_std = np.std(LEYE, axis=0)
np.save("mean_std/leye_mean.npy", leye_mean)
np.save("mean_std/leye_std.npy", leye_std)
del LEYE, leye_mean, leye_std
_ = gc.collect()

print("## processing reye")
for rhand, lhand, rpose, lpose, lip, nose, leye, reye in tqdm(gen(df)):
    REYE.extend(reye)
REYE = np.array(REYE)
reye_mean = np.mean(REYE, axis=0)
reye_std = np.std(REYE, axis=0)
np.save("mean_std/reye_mean.npy", reye_mean)
np.save("mean_std/reye_std.npy", reye_std)
del REYE, reye_mean, reye_std
_ = gc.collect()

## processing rhand


57497it [06:38, 144.26it/s]


## processing lhand


57497it [03:36, 266.07it/s]


## processing rpose


...

57497it [04:53, 196.14it/s]


## processing leye


57497it [03:21, 285.48it/s]


## processing reye


57497it [05:10, 185.25it/s]


In [7]:
@tf.function(jit_compile=True)
def pre_process0(x):
    lip_x = tf.gather(x, LIP_IDX_X, axis=1)
    lip_y = tf.gather(x, LIP_IDX_Y, axis=1)
    lip_z = tf.gather(x, LIP_IDX_Z, axis=1)
    
    nose_x = tf.gather(x, NOSE_IDX_X, axis=1)
    nose_y = tf.gather(x, NOSE_IDX_Y, axis=1)
    nose_z = tf.gather(x, NOSE_IDX_Z, axis=1)
    
    leye_x = tf.gather(x, LEYE_IDX_X, axis=1)
    leye_y = tf.gather(x, LEYE_IDX_Y, axis=1)
    leye_z = tf.gather(x, LEYE_IDX_Z, axis=1)
    
    reye_x = tf.gather(x, REYE_IDX_X, axis=1)
    reye_y = tf.gather(x, REYE_IDX_Y, axis=1)
    reye_z = tf.gather(x, REYE_IDX_Z, axis=1)
    
    rhand_x = tf.gather(x, RHAND_IDX_X, axis=1)
    rhand_y = tf.gather(x, RHAND_IDX_Y, axis=1)
    rhand_z = tf.gather(x, RHAND_IDX_Z, axis=1)
    
    lhand_x = tf.gather(x, LHAND_IDX_X, axis=1)
    lhand_y = tf.gather(x, LHAND_IDX_Y, axis=1)
    lhand_z = tf.gather(x, LHAND_IDX_Z, axis=1)

    rpose_x = tf.gather(x, RPOSE_IDX_X, axis=1)
    rpose_y = tf.gather(x, RPOSE_IDX_Y, axis=1)
    rpose_z = tf.gather(x, RPOSE_IDX_Z, axis=1)
    
    lpose_x = tf.gather(x, LPOSE_IDX_X, axis=1)
    lpose_y = tf.gather(x, LPOSE_IDX_Y, axis=1)
    lpose_z = tf.gather(x, LPOSE_IDX_Z, axis=1)
    
    lip   = tf.concat([lip_x[..., tf.newaxis], lip_y[..., tf.newaxis], lip_z[..., tf.newaxis]], axis=-1)
    nose  = tf.concat([nose_x[..., tf.newaxis], nose_y[..., tf.newaxis], nose_z[..., tf.newaxis]], axis=-1)
    leye  = tf.concat([leye_x[..., tf.newaxis], leye_y[..., tf.newaxis], leye_z[..., tf.newaxis]], axis=-1)
    reye  = tf.concat([reye_x[..., tf.newaxis], reye_y[..., tf.newaxis], reye_z[..., tf.newaxis]], axis=-1)
    rhand = tf.concat([rhand_x[..., tf.newaxis], rhand_y[..., tf.newaxis], rhand_z[..., tf.newaxis]], axis=-1)
    lhand = tf.concat([lhand_x[..., tf.newaxis], lhand_y[..., tf.newaxis], lhand_z[..., tf.newaxis]], axis=-1)
    rpose = tf.concat([rpose_x[..., tf.newaxis], rpose_y[..., tf.newaxis], rpose_z[..., tf.newaxis]], axis=-1)
    lpose = tf.concat([lpose_x[..., tf.newaxis], lpose_y[..., tf.newaxis], lpose_z[..., tf.newaxis]], axis=-1)
    
    hand = tf.concat([rhand, lhand], axis=1)
    hand = tf.where(tf.math.is_nan(hand), 0.0, hand)
    mask = tf.math.not_equal(tf.reduce_sum(hand, axis=[1, 2]), 0.0)

    lip = lip[mask]
    nose = nose[mask]
    leye = leye[mask]
    reye = reye[mask]
    rhand = rhand[mask]
    lhand = lhand[mask]
    rpose = rpose[mask]
    lpose = lpose[mask]

    return lip, nose, leye, reye, rhand, lhand, rpose, lpose

pre_process0(frames)

(<tf.Tensor: shape=(58, 40, 3), dtype=float32, numpy=
 array([[[ 6.4492857e-01,  5.4160029e-01,  8.9848827e-04],
         [ 6.5078962e-01,  5.3814346e-01, -1.2366068e-02],
         [ 6.5999925e-01,  5.3419906e-01, -2.7644193e-02],
         ...,
         [ 7.5821137e-01,  5.3899992e-01, -3.8036082e-02],
         [ 7.6887351e-01,  5.3971463e-01, -3.1674828e-02],
         [ 7.8060400e-01,  5.4074234e-01, -2.7605090e-02]],
 
        [[ 6.4930332e-01,  5.4280776e-01,  2.6618475e-03],
         [ 6.5443867e-01,  5.3907025e-01, -9.8490389e-03],
         [ 6.6302049e-01,  5.3474104e-01, -2.4537543e-02],
         ...,
         [ 7.5848484e-01,  5.3844941e-01, -3.5687264e-02],
         [ 7.6911563e-01,  5.3966576e-01, -2.9702550e-02],
         [ 7.8035623e-01,  5.4101783e-01, -2.5644979e-02]],
 
        [[ 6.5260077e-01,  5.4284489e-01, -1.7653322e-03],
         [ 6.5799290e-01,  5.3890228e-01, -1.4589361e-02],
         [ 6.6648668e-01,  5.3446651e-01, -2.9142667e-02],
         ...,
         [ 7.

In [8]:
def load_relevant_data_subset(pq_path):
    return pd.read_parquet(pq_path, columns=SEL_COLS)

if not os.path.isdir("tfds"): os.mkdir("tfds")

for file_id in tqdm(df.file_id.unique()):
    pqfile = f"{inpdir}/{file_id}.parquet"
    tffile = f"tfds/{file_id}.tfrecord"
    seq_refs = df.loc[df.file_id == file_id]
    seqs = load_relevant_data_subset(pqfile)
    
    with tf.io.TFRecordWriter(tffile) as file_writer:
        for seq_id, phrase in zip(seq_refs.sequence_id, seq_refs.phrase):
            frames = seqs.iloc[seqs.index == seq_id].to_numpy()
            
            lip, nose, leye, reye, rhand, lhand, rpose, lpose = pre_process0(frames)

#             if max(rhand.shape[0], lhand.shape[0]) < len(phrase):
#                 continue
                
            features = {}
            features["lip"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(lip, -1).numpy())) 
            features["nose"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(nose, -1).numpy())) 
            features["leye"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(leye, -1).numpy())) 
            features["reye"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(reye, -1).numpy())) 
            features["rhand"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(rhand, -1).numpy())) 
            features["lhand"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(lhand, -1).numpy())) 
            features["rpose"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(rpose, -1).numpy())) 
            features["lpose"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(lpose, -1).numpy())) 
            features["phrase"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[char_to_num[x] for x in phrase]))
            
            record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()
            file_writer.write(record_bytes)
        file_writer.close()

100%|██████████| 68/68 [1:03:09<00:00, 55.73s/it]


In [9]:
def decode_fn(record_bytes):
    schema = {
        "lip": tf.io.VarLenFeature(tf.float32),
        "nose": tf.io.VarLenFeature(tf.float32),
        "leye": tf.io.VarLenFeature(tf.float32),
        "reye": tf.io.VarLenFeature(tf.float32),
        "rhand": tf.io.VarLenFeature(tf.float32),
        "lhand": tf.io.VarLenFeature(tf.float32),
        "rpose": tf.io.VarLenFeature(tf.float32),
        "lpose": tf.io.VarLenFeature(tf.float32),
        "phrase": tf.io.VarLenFeature(tf.int64)
    }
    x = tf.io.parse_single_example(record_bytes, schema)

    lip = tf.reshape(tf.sparse.to_dense(x["lip"]), (-1, 40, 3))
    nose = tf.reshape(tf.sparse.to_dense(x["nose"]), (-1, 4, 3))
    leye = tf.reshape(tf.sparse.to_dense(x["leye"]), (-1, 16, 3))
    reye = tf.reshape(tf.sparse.to_dense(x["reye"]), (-1, 16, 3))
    rhand = tf.reshape(tf.sparse.to_dense(x["rhand"]), (-1, 21, 3))
    lhand = tf.reshape(tf.sparse.to_dense(x["lhand"]), (-1, 21, 3))
    rpose = tf.reshape(tf.sparse.to_dense(x["rpose"]), (-1, 5, 3))
    lpose = tf.reshape(tf.sparse.to_dense(x["lpose"]), (-1, 5, 3))
    phrase = tf.sparse.to_dense(x["phrase"])

    return lip, nose, leye, reye, rhand, lhand, rpose, lpose, phrase

    
tffiles = [f"tfds/{file_id}.tfrecord" for file_id in df.file_id.unique()]
for batch in tf.data.TFRecordDataset(tffiles).map(decode_fn).take(1):
    print(batch[0].shape, batch[1].shape, batch[2].shape, batch[3].shape, batch[4].shape, batch[5].shape, batch[6].shape, batch[7].shape, batch[8])

(58, 40, 3) (58, 4, 3) (58, 16, 3) (58, 16, 3) (58, 21, 3) (58, 21, 3) (58, 5, 3) (58, 5, 3) tf.Tensor([18  0 34 49 36 36 42 39 46 52 50 36], shape=(12,), dtype=int64)
